<a href="https://colab.research.google.com/github/phelipecustodio/public/blob/main/Concess%C3%A3o_de_Cr%C3%A9dito_com_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [12]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 4  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "20.163.207.167" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}

#@markdown Informe o seu nome:
students_name = "Phelipe Custodio Lima" #@param {type:"string"}

#@markdown Ah, e não se esqueça de executar esta célula!


In [13]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [14]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [15]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
11936,"Felicity, Sarinana",1578.868653,42.107425,1,1,1,0,1,1
6124,"Mutasim, al-Shakir",3988.275257,34.330977,0,0,1,0,1,3
20250,"Jackson, Schaeffer",3952.113639,32.588803,0,0,0,0,1,2
23359,"James, Geyer",4402.445632,60.563534,0,0,1,0,1,2


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [16]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [17]:
response01.text.encode('utf8').decode()

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [18]:
response01 = requests.request("POST", url + f"?model=modelo01&aluno=Phelipe Custodio Lima", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [1, 0, 0, 0],
 'proba': [[0.0007538729695057977, 0.9992461270304942],
  [0.9880161436022514, 0.011983856397748571],
  [0.8841946578017753, 0.11580534219822465],
  [0.9993429010122433, 0.0006570989877566777]]}

In [19]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [20]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [1.0000874284098702,
  -0.016999743839622248,
  0.28948450480857146,
  -0.0031035858535341555]}

In [21]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [22]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
11936,"Felicity, Sarinana",1,"[0.0007538729695057977, 0.9992461270304942]",1.000087
6124,"Mutasim, al-Shakir",0,"[0.9880161436022514, 0.011983856397748571]",-0.017000
20250,"Jackson, Schaeffer",0,"[0.8841946578017753, 0.11580534219822465]",0.289485
23359,"James, Geyer",0,"[0.9993429010122433, 0.0006570989877566777]",-0.003104
